## Plotting the Albedo Products

### Quicklook a single GeoTIFF (one date)

In [ ]:
import os
import sys
import glob
import math
import folium
import pathlib
import zipfile
import numpy as np
import xarray as xr
import pandas as pd
import leafmap as lm  # ipyleaflet backend
from PIL import Image
import rasterio as rio
import geopandas as gpd
from matplotlib import cm
from jinja2 import Template
import branca.colormap as bcm
import matplotlib.pyplot as plt
from matplotlib import cm, colors
import leafmap.foliumap as leafmap
from matplotlib import cm as mpl_cm
from folium.elements import MacroElement
from rasterio.plot import plotting_extent
from rasterio.warp import transform_bounds
from branca.colormap import LinearColormap
import ipywidgets, ipyleaflet, jupyterlab_widgets
from pystac.extensions.eo import EOExtension as eo
from rasterio import warp, windows, features, enums
print("ipywidgets:", ipywidgets.__version__, " ipyleaflet:", ipyleaflet.__version__)

plt.style.use("~/geoscience/albedo_downscaling/MNRAS.mplstyle")
%matplotlib inline

In [ ]:
tif = "/bsuhome/tnde/scratch/felix/Sentinel-2/s2_albedo_outputs/2021-09-03_S2_BSA20m_SW_hard.tif"  # <- change date/variant
with rio.open(tif) as ds:
    arr = ds.read(1, masked=True)  # nodata -> mask
    extent = plotting_extent(ds)

plt.figure(figsize=(8,6))
im = plt.imshow(arr, extent=extent, origin="upper")# , vmin=0, vmax=1)
plt.tick_params(axis='x', rotation=45)
plt.title("S2 blue-sky SW albedo (keep-all)")
plt.xlabel(ds.crs.to_string()); plt.ylabel("y (map units)")
cbar = plt.colorbar(im, fraction=0.046, pad=0.04)
cbar.set_label("Albedo (0–1)")
plt.tight_layout(); plt.show()

### Side-by-side: keep-snow vs snow-only (same date)

In [ ]:
# date = "2021-09-18"
date = "2021-11-27"
# date = "2021-11-02"
# date = "2021-11-07"
# date = "2021-12-07"
root = "/bsuhome/tnde/scratch/felix/Sentinel-2/s2_albedo_outputs"
# a = f"{date}_albedoSW_keepAll_hard.tif"
# b = f"{date}_albedoSW_snowOnly_hard.tif"

a = f"{date}_S2_BSA20m_SW_hard.tif"
b = f"{date}_S2_BSA20m_snowOnly_hard.tif"

fig, axs = plt.subplots(1, 2, figsize=(12,5), constrained_layout=True)
for ax, path, title in zip(axs, [f"{root}/{a}", f"{root}/{b}"], ["keep-all", "snow-only"]):
    with rio.open(path) as ds:
        arr = ds.read(1, masked=True)
        extent = plotting_extent(ds)
    im = ax.imshow(arr, extent=extent, origin="upper") # , vmin=0, vmax=1)
    ax.tick_params(axis='x', rotation=0)
    ax.set_title(f"{date} • SW albedo • {title}")
    ax.set_xlabel(ds.crs.to_string()); ax.set_ylabel("")

fig.colorbar(im, ax=axs, location="right", fraction=0.046, pad=0.04, label="Albedo (0–1)")
plt.show()

### Small multiples over many dates (GeoTIFFs)

In [ ]:
# files = sorted(glob.glob("/bsuhome/tnde/scratch/felix/Sentinel-2/s2_albedo_outputs/*_albedoSW_keepAll_hard.tif"))  # pick a product/variant
files = sorted(glob.glob("/bsuhome/tnde/scratch/felix/Sentinel-2/s2_albedo_outputs/*_S2_BLUE20m_SW_hard.tif"))  # pick a product/variant
n = len(files)
cols = 3
rows = math.ceil(n/cols)

fig, axes = plt.subplots(rows, cols, figsize=(4*cols, 3.5*rows), constrained_layout=True)
axes = axes.ravel()

for ax, f in zip(axes, files):
    with rio.open(f) as ds:
        arr = ds.read(1, masked=True)
        extent = plotting_extent(ds)
    im = ax.imshow(arr, extent=extent, origin="upper") # , vmin=0, vmax=1)
    ax.tick_params(axis='x', rotation=0)
    ax.set_title(f.split("/")[-1].split("_")[0])  # date from filename
    ax.set_xticks([]); ax.set_yticks([])

# hide any extra subplots
for j in range(len(files), len(axes)):
    axes[j].axis("off")

fig.colorbar(im, ax=axes.tolist(), fraction=0.015, pad=0.02, label="Albedo (0–1)")
plt.show()


### Use the NetCDF stack (fastest for time series and faceting)

#### a) Plot one date from the stack

In [ ]:
# ds = xr.open_mfdataset(
#     ["/bsuhome/tnde/scratch/felix/Sentinel-2/s2_albedo_outputs/albedo_stack1.nc", 
#      "/bsuhome/tnde/scratch/felix/Sentinel-2/s2_albedo_outputs/albedo_stack2.nc", 
#      "/bsuhome/tnde/scratch/felix/Sentinel-2/s2_albedo_outputs/albedo_stack3.nc"],
#     combine="by_coords")

# if "time" in ds.coords:
#     ds = ds.sortby("time").drop_duplicates("time")

# # Save to one file
# ds.to_netcdf("/bsuhome/tnde/scratch/felix/Sentinel-2/s2_albedo_outputs/albedo_stack_merged.nc")


In [ ]:
ds = xr.open_dataset("/bsuhome/tnde/scratch/felix/Sentinel-2/s2_albedo_outputs/albedo_stack_merged.nc")

# Check duplicates
time_index = ds.indexes["time"]              # pandas.DatetimeIndex
print("Unique?", time_index.is_unique)
print("Num duplicates:", time_index.duplicated().sum())

# Drop duplicate time stamps (keep first) and sort by time
dsu = ds.isel(time=~time_index.duplicated()).sortby("time")

# Now nearest selection works
date = np.datetime64("2021-09-08")
da = dsu["albedo_sw_keep_hard"].sel(time=date, method="nearest").squeeze(drop=True)

# plt.figure(figsize=(8,6))
fig, axes = plt.subplots(figsize=(8,6), constrained_layout=True)
im = plt.imshow(np.ma.filled(da, np.nan), extent=extent, origin="upper") #, vmin=0, vmax=1)
last_im = im
plt.title(f"Shortwave albedo (keep-all) - {str(da['time'].dt.date.values)[:19]}")
plt.tick_params(axis='x', rotation=45)
# colorbar 
cbar = fig.colorbar(last_im, ax=axes, fraction=0.046, pad=0.04)
cbar.set_label("Albedo (0–1)")
plt.show()


In [ ]:
ds = xr.open_dataset("/bsuhome/tnde/scratch/felix/Sentinel-2/s2_albedo_outputs/albedo_stack_merged.nc")
date = "2021-09-08"

# Boolean mask of the time coordinate
mask = ds.time.dt.strftime("%Y-%m-%d") == date

# OPTION 1: boolean -> positions -> isel
idx = np.nonzero(mask.values)[0]
day_stack = ds["albedo_sw_keep_hard"].isel(time=idx)

# OPTION 2 (equivalent): keep by mask with drop=True
# day_stack = ds["albedo_sw_keep"].where(mask, drop=True)

# Aggregate across all observations for that day
da = day_stack.mean("time", skipna=True)

# plt.figure(figsize=(8,6))
fig, axes = plt.subplots(figsize=(8,6), constrained_layout=True)
# da.plot.imshow(vmin=0, vmax=1, add_colorbar=True)
im = plt.imshow(np.ma.filled(da, np.nan), extent=extent, origin="upper") #, vmin=0, vmax=1)
last_im = im
plt.title(date)
plt.tick_params(axis='x', rotation=45)

# colorbar
cbar = fig.colorbar(last_im, ax=axes, fraction=0.046, pad=0.04)
cbar.set_label("Albedo (0–1)")
plt.show()


In [ ]:
time_index = ds.indexes["time"]
dsu = ds.isel(time=~time_index.duplicated()).sortby("time")

da = dsu["albedo_sw_keep_hard"].sel(time=np.datetime64(date), method="nearest").squeeze(drop=True)

# plt.figure(figsize=(8,6))
fig, axes = plt.subplots(figsize=(8,6), constrained_layout=True)
# da.plot.imshow(vmin=0, vmax=1, add_colorbar=True)
im = plt.imshow(np.ma.filled(da, np.nan), extent=extent, origin="upper") #, vmin=0, vmax=1)
last_im = im
plt.title(date)
plt.tick_params(axis='x', rotation=45)

# colorbar
cbar = fig.colorbar(last_im, ax=axes, fraction=0.046, pad=0.04)
cbar.set_label("Albedo (0–1)")
plt.show()

In [ ]:
mask = (ds.time.dt.floor("D") == np.datetime64(date))
idx = np.flatnonzero(mask.values)   # integer positions
day_stack = ds["albedo_sw_keep_hard"].isel(time=idx)  # positional indexing (no uniqueness needed)

# Aggregate over the day if you want a single raster
da_daily_mean = day_stack.mean("time", skipna=True)
# plt.figure(figsize=(8,6))
fig, axes = plt.subplots(figsize=(8,6), constrained_layout=True)
# da_daily_mean.plot.imshow(vmin=0, vmax=1, add_colorbar=True)
im = plt.imshow(np.ma.filled(da_daily_mean, np.nan), extent=extent, origin="upper") #, vmin=0, vmax=1)
last_im = im
plt.title(f"{date} (daily mean)")
plt.tick_params(axis='x', rotation=45)

# colorbar
cbar = fig.colorbar(last_im, ax=axes, fraction=0.046, pad=0.04)
cbar.set_label("Albedo (0–1)")
plt.show()

In [ ]:
# See what timestamps you have
ds = xr.open_dataset("/bsuhome/tnde/scratch/felix/Sentinel-2/s2_albedo_outputs/albedo_stack_merged.nc")
print(pd.to_datetime(ds.time.values))


#### c) AOI-mean time series

In [ ]:
mean_ts = ds["albedo_sw_keep_hard"].mean(dim=("y","x"), skipna=True)
plt.subplots(figsize=(10, 4))
mean_ts.plot(marker="o")
plt.title("AOI mean shortwave albedo (keep-all)")
plt.ylabel("Albedo (0–1)")
plt.show()


### d) Compare VIS, NIR, SW over time

In [ ]:
sw = ds["albedo_sw_keep_hard"].mean(("y","x"), skipna=True)
vi = ds["albedo_vis_keep_hard"].mean(("y","x"), skipna=True)
ni = ds["albedo_nir_keep_hard"].mean(("y","x"), skipna=True)

plt.figure(figsize=(10,4))
plt.plot(sw["time"], sw, label="SW")
plt.plot(vi["time"], vi, label="VIS")
plt.plot(ni["time"], ni, label="NIR")
plt.legend(); plt.ylabel("AOI mean albedo"); plt.xlabel("Date")
plt.title("VIS / NIR / SW (keep-all) — AOI mean")
# Rotate x tick labels by some degrees
plt.tick_params(axis="x", labelrotation=45)
plt.tight_layout(); plt.show()


In [ ]:
# Load
ds = xr.open_dataset("/bsuhome/tnde/scratch/felix/Sentinel-2/s2_albedo_outputs/albedo_stack_merged.nc")

def mean_series(ds, name):
    return ds[name].sortby("time").mean(("y","x"), skipna=True) if name in ds else None

# --- Original NTB products: keep-snow (solid) and snow-only (dashed) ---
sw_k = mean_series(ds, "albedo_sw_keep_hard")
vi_k = mean_series(ds, "albedo_vis_keep_hard")
ni_k = mean_series(ds, "albedo_nir_keep_hard")

sw_s = mean_series(ds, "albedo_sw_snow_hard")
vi_s = mean_series(ds, "albedo_vis_snow_hard")
ni_s = mean_series(ds, "albedo_nir_snow_hard")

# --- BRDF products: keep-snow and snow-only ---
bsa_k = mean_series(ds, "bsa_sw_keep_hard")
wsa_k = mean_series(ds, "wsa_sw_keep_hard")
blue_k = mean_series(ds, "blue_sw_keep_hard")

bsa_s = mean_series(ds, "bsa_sw_snow_hard")
wsa_s = mean_series(ds, "wsa_sw_snow_hard")
blue_s = mean_series(ds, "blue_sw_snow_hard")

# --- Snow cover fraction over total number of pixels ---
snow_frac = (~np.isnan(ds["albedo_sw_snow_hard"])).mean(("y","x"))


# sharey=True makes both subplots share the same y-axis
fig, axes = plt.subplots(2, 1, figsize=(12,10), sharex=True, sharey=False)

# ---------------- Left: VIS / NIR / SW (keep-snow vs snow-only) ----------------
ax = axes[0]

# Plot keep-snow (solid) and capture colors
lines = {}
if sw_k is not None: lines["SW"]  = ax.plot(sw_k["time"], sw_k,  label="SW (keep-all)",  linewidth=2)[0]
if vi_k is not None: lines["VIS"] = ax.plot(vi_k["time"], vi_k, label="VIS (keep-all)", linewidth=2)[0]
if ni_k is not None: lines["NIR"] = ax.plot(ni_k["time"], ni_k, label="NIR (keep-all)", linewidth=2)[0]

# Snow-only (dashed, same color as keep)
if sw_s is not None and "SW" in lines:
    ax.plot(sw_s["time"], sw_s, "--", color=lines["SW"].get_color(),  label="SW (snow-only)")
if vi_s is not None and "VIS" in lines:
    ax.plot(vi_s["time"], vi_s, "--", color=lines["VIS"].get_color(), label="VIS (snow-only)")
if ni_s is not None and "NIR" in lines:
    ax.plot(ni_s["time"], ni_s, "--", color=lines["NIR"].get_color(), label="NIR (snow-only)")

# Plot snow fraction    
ax.plot(snow_frac["time"], snow_frac, linestyle = "--", label="Total snow frac")

ax.set_ylabel("AOI mean albedo")
ax.set_xlabel("Date")
ax.set_title("keep-all vs snow-only")
ax.grid(True, alpha=0.3)
ax.legend(ncol=1, fontsize=6)

# Rotate x tick labels 45 degrees
ax.tick_params(axis="x", labelrotation=60)

# ---------------- Right: BSA / WSA / BLUE (keep-snow vs snow-only) ---------------
ax = axes[1]

lines_b = {}
if bsa_k is not None: lines_b["BSA"] = ax.plot(bsa_k["time"], bsa_k, label="BSA (keep-all)", linewidth=2)[0]
if wsa_k is not None: lines_b["WSA"] = ax.plot(wsa_k["time"], wsa_k, label="WSA (keep-all)", linewidth=2)[0]
if blue_k is not None: lines_b["BLUE"] = ax.plot(blue_k["time"], blue_k, label="BLUE (keep-all)", linewidth=2)[0]

if bsa_s is not None and "BSA" in lines_b:
    ax.plot(bsa_s["time"], bsa_s, "--", color=lines_b["BSA"].get_color(), label="BSA (snow-only)")
if wsa_s is not None and "WSA" in lines_b:
    ax.plot(wsa_s["time"], wsa_s, "--", color=lines_b["WSA"].get_color(), label="WSA (snow-only)")
if blue_s is not None and "BLUE" in lines_b:
    ax.plot(blue_s["time"], blue_s, "--", color=lines_b["BLUE"].get_color(), label="BLUE (snow-only)")
    
# Plot snow fraction    
ax.plot(snow_frac["time"], snow_frac, linestyle = "--", label="Total snow frac")

# ax.set_ylabel("AOI mean albedo")
ax.set_xlabel("Date")
ax.set_title("keep-all vs snow-only")
ax.grid(True, alpha=0.3)
ax.legend(ncol=1, fontsize=6)

# Rotate x tick labels by some degrees
ax.tick_params(axis="x", labelrotation=60)

plt.tight_layout(h_pad=0.0, w_pad=-1.10)
# plt.tight_layout()
plt.show()


In [ ]:
# Load
ds = xr.open_dataset("/bsuhome/tnde/scratch/felix/Sentinel-2/s2_albedo_outputs/albedo_stack_merged.nc")

def mean_series(ds, name):
    return ds[name].sortby("time").mean(("y","x"), skipna=True) if name in ds else None

# --- Original NTB products ---
sw_k = mean_series(ds, "albedo_sw_keep_hard")
vi_k = mean_series(ds, "albedo_vis_keep_hard")
ni_k = mean_series(ds, "albedo_nir_keep_hard")

sw_s = mean_series(ds, "albedo_sw_snow_hard")
vi_s = mean_series(ds, "albedo_vis_snow_hard")
ni_s = mean_series(ds, "albedo_nir_snow_hard")

# --- BRDF products ---
bsa_k  = mean_series(ds, "bsa_sw_keep_hard")
wsa_k  = mean_series(ds, "wsa_sw_keep_hard")
blue_k = mean_series(ds, "blue_sw_keep_hard")   # or "blue_sw_keep" 

bsa_s  = mean_series(ds, "bsa_sw_snow_hard")
wsa_s  = mean_series(ds, "wsa_sw_snow_hard")
blue_s = mean_series(ds, "blue_sw_snow_hard")   # or "blue_sw_snow"

# --- Snow cover fraction over total number of pixels ---
snow_frac = (~np.isnan(ds["albedo_sw_snow_hard"])).mean(("y","x"))

fig, axes = plt.subplots(2, 1, figsize=(12,10), sharex=True, sharey=False)

# ---------------- Left: ALL KEEP-SNOW ----------------
ax_keep = axes[0]
keep_lines = {}

def plot_keep(series, label):
    if series is not None:
        keep_lines[label] = ax_keep.plot(series["time"], series, label=label, linewidth=2)[0]

plot_keep(sw_k,   "SW (keep)")
plot_keep(vi_k,   "VIS (keep)")
plot_keep(ni_k,   "NIR (keep)")
plot_keep(bsa_k,  "BSA (keep)")
plot_keep(wsa_k,  "WSA (keep)")
plot_keep(blue_k, "BLUE (keep)")
plot_keep(snow_frac, "Total snow frac")

ax_keep.set_ylabel("AOI mean albedo")
ax_keep.set_xlabel("Date")
ax_keep.set_title("Keep-all")
ax_keep.grid(True, alpha=0.3)
ax_keep.legend(ncol=1, fontsize=6)

# Rotate x tick labels by some degrees
ax_keep.tick_params(axis="x", labelrotation=60)

# ---------------- Right: ALL SNOW-ONLY ----------------
ax_snow = axes[1]

def plot_snow(series, label_base):
    if series is None:
        return
    # match color with the corresponding keep series if it exists
    keep_label = f"{label_base} (keep)"
    color = keep_lines[keep_label].get_color() if keep_label in keep_lines else None
    ax_snow.plot(series["time"], series, label=f"{label_base} (snow-only)", linewidth=2, color=color)

plot_snow(sw_s,   "SW")
plot_snow(vi_s,   "VIS")
plot_snow(ni_s,   "NIR")
plot_snow(bsa_s,  "BSA")
plot_snow(wsa_s,  "WSA")
plot_snow(blue_s, "BLUE")
plot_snow(snow_frac, "Total snow frac")

# ax_snow.set_ylabel("AOI mean albedo")
ax_snow.set_xlabel("Date")
ax_snow.set_title("Snow-only")
ax_snow.grid(True, alpha=0.3)
ax_snow.legend(ncol=1, fontsize=6)

# Rotate x tick labels by some degrees
ax_snow.tick_params(axis="x", labelrotation=60)

plt.tight_layout(h_pad=0.0, w_pad=-1.10)
plt.show()

### Snow Cover Fraction

In [ ]:
def snow_fraction(ds, keep_var="albedo_sw_keep_hard", snow_var="albedo_sw_snow_hard"):
    """Fraction of valid pixels that are snow: valid(snow-only) / valid(keep)."""
    if keep_var in ds and snow_var in ds:
        keep_valid = ds[keep_var].notnull().sum(("y","x"))
        snow_valid = ds[snow_var].notnull().sum(("y","x"))
        frac = snow_valid / keep_valid#.where(keep_valid > 0)).sortby("time")
        return frac
    return None

# Snow fraction series (single series; show on both subplots)
sf = snow_fraction(ds, "albedo_sw_keep_hard", "albedo_sw_snow_hard")  # uses SW masks; change if you prefer VIS/NIR

plt.subplots(figsize=(10, 4))
sf_line = plt.plot(sf["time"], sf, marker="o", markersize=3, color="k", linestyle="--", label="Snow fraction over valid pixels")[0]

snow_frac = (~np.isnan(ds["albedo_sw_snow_hard"])).mean(("y","x"))
snow_frac.plot(marker="*", markersize=3, label="Snow fraction over all pixels"); 
plt.title("Snow-covered fraction"); 
plt.ylabel("Fraction")
plt.xlabel("Date")
plt.legend(ncol=1, fontsize=6)
plt.show()


In [ ]:
ds = xr.open_dataset("/bsuhome/tnde/scratch/felix/Sentinel-2/s2_albedo_outputs/albedo_stack_merged.nc")

def snow_fraction_from_ds(ds, keep_var="albedo_sw_keep_hard", snow_var="albedo_sw_snow_hard", pixel_size_m=20):
    # valid pixel counts per time
    n_keep = ds[keep_var].notnull().sum(("y","x"))
    n_snow = ds[snow_var].notnull().sum(("y","x"))

    # total pixel count in the grid (constant across time)
    total_px = ds[keep_var].sizes["y"] * ds[keep_var].sizes["x"]

    # fractions and areas
    frac_valid = (n_snow / n_keep.where(n_keep > 0)).rename("snow_fraction_valid_px")          # 0..1 of valid pixels
    frac_total = (n_snow / total_px).rename("snow_fraction_total_px")                    # 0..1 of all pixels

    px_area_km2 = (pixel_size_m**2) / 1e6
    snow_area_km2 = (n_snow * px_area_km2).rename("snow_area_km2")
    aoi_area_km2  = (n_keep * px_area_km2).rename("aoi_area_km2")

    return xr.Dataset({
        "snow_fraction_valid_px": frac_valid,
        "snow_fraction_total_px": frac_total,
        "snow_area_km2": snow_area_km2,
        "aoi_area_km2": aoi_area_km2,
    }).sortby("time")

sf_ds = snow_fraction_from_ds(ds)      # uses SW masks by default
print(sf_ds)

# Save to CSV (time, fraction, areas)
sf_ds.to_dataframe().to_csv("/bsuhome/tnde/scratch/felix/Sentinel-2/s2_albedo_outputs/snow_fraction_timeseries.csv")

plt.subplots(figsize=(10, 4))
# Snow fraction: valid_snow_px/total_valid_px
sf_ds["snow_fraction_valid_px"].plot(marker="o", color="r")
# sf_line = plt.plot(sf["time"], sf, color="r", linestyle="-", label="Over Valid Pixel")[0]

# Snow fraction: valid_snow_px/total_px
sf_ds["snow_fraction_total_px"].plot(marker="o", markersize=3, linestyle="--", color="k", label="Over valid pixel")
snow_frac = (~np.isnan(ds["albedo_sw_snow_hard"])).mean(("y","x"))
snow_frac.plot(marker="o", label="Over all pixels"); 
plt.title("Snow-covered fraction"); 
plt.legend(ncol=1, fontsize=6)
plt.show()


### Save Snow Fraction and Area to CSV

In [ ]:
# tiff_dir = "/bsuhome/tnde/scratch/felix/Sentinel-2/s2_albedo_outputs"
# keep_files = sorted(glob.glob(os.path.join(tiff_dir, "*_albedoSW_keepAll_hard.tif")))
# snow_lookup = {os.path.basename(f).split("_")[0]: f
#                for f in glob.glob(os.path.join(tiff_dir, "*_albedoSW_snowOnly_hard.tif"))}

# rows = []
# for keep_path in keep_files:
#     date = os.path.basename(keep_path).split("_")[0]
#     snow_path = snow_lookup.get(date)

#     with rio.open(keep_path) as dk:
#         keep = dk.read(1, masked=True).filled(np.nan)
#         H, W = keep.shape
#         total_px = H * W                               # <-- total pixels in this raster
#         px_area_m2 = abs(dk.transform.a * dk.transform.e)

#     n_keep = int(np.isfinite(keep).sum())
#     n_snow = 0
#     if snow_path:
#         with rio.open(snow_path) as ds_s:
#             snow = ds_s.read(1, masked=True).filled(np.nan)
#             n_snow = int(np.isfinite(snow).sum())

#     snow_fraction_valid = (n_snow / n_keep) if n_keep > 0 else np.nan
#     snow_fraction_total = n_snow / total_px           # <-- new column

#     rows.append({
#         "date": date,
#         "snow_fraction": snow_fraction_valid,         # n_snow / valid (keep-all) pixels
#         "snow_fraction_total": snow_fraction_total,   # n_snow / total pixels
#         "snow_area_km2": (n_snow * px_area_m2) / 1e6,
#         "aoi_area_km2":  (n_keep * px_area_m2) / 1e6,
#         "n_snow_px": n_snow,
#         "n_keep_px": n_keep,
#         "total_px": total_px,
#     })

# df = pd.DataFrame(rows).sort_values("date")
# print(df.head())
# df.to_csv(os.path.join(tiff_dir, "snow_fraction_from_tifs.csv"), index=False)


### Cloud Cover Fraction


The Sentinel-2 Scene Classification Layer (SCL) has discrete codes 0–11. Here’s the official mapping from ESA:


| Code | Class Description                     |
| ---- | ------------------------------------- |
| 0    | No Data                               |
| 1    | Saturated or defective                |
| 2    | Dark area pixels                      |
| 3    | Cloud shadows                         |
| 4    | Vegetation                            |
| 5    | Bare soils                            |
| 6    | Water                                 |
| 7    | Clouds low probability / Unclassified |
| 8    | Clouds medium probability             |
| 9    | Clouds high probability               |
| 10   | Thin cirrus                           |
| 11   | Snow or ice                           |


In [ ]:
ds = xr.open_dataset("/bsuhome/tnde/scratch/felix/Sentinel-2/s2_albedo_outputs/albedo_stack_merged.nc")

# total pixels per scene
total_px = ds["albedo_sw_keep_hard"].sizes["y"] * ds["albedo_sw_keep_hard"].sizes["x"]
valid_px = ds["albedo_sw_keep_hard"].notnull().sum(("y","x"))     # kept (non-cloud etc.)
masked_frac = 1.0 - (valid_px / total_px)                    # ≈ cloud+other-masks

mf = masked_frac.sortby("time").to_series()

fig, ax = plt.subplots(figsize=(10,4))
ax.plot(mf.index, mf.values, marker="o", markersize=3, label="Masked fraction") # approx cloudiness
ax.set_ylim(-0.1, 1.1); ax.set_ylabel("Cloud fraction"); ax.set_xlabel("Date")
# ax.grid(True, alpha=0.3); ax.legend(); ax.set_title("Approximate cloudiness")# from keep snow mask
plt.tight_layout(); plt.show()

In [ ]:
function_path = os.path.expanduser("~/geoscience/albedo_downscaling/sentinel_2/s2_functions")
sys.path.append(function_path)
# import all the helper functions.
from s2_20m_download_final import *  

# ----------------------------
# Planetary Computer STAC
# ----------------------------
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

# Search Sentinel-2 L2A
search = catalog.search(
    collections=["sentinel-2-l2a"],
    intersects=area_of_interest,
    datetime=time_of_interest,
    # query={"eo:cloud_cover": {"lt": 60}}, # up to 60% cloud cover
)

# New:
items = list(search.items())
print(f"Returned {len(items)} items")

if not items:
    raise SystemExit("No items found for query.")

for item in items:
    date = item.datetime.date().isoformat()
    # print(f"Processing {item.id} ({date})")


In [ ]:
CLOUD_CLASSES = np.array([8, 9, 10], dtype=np.uint8)     # clouds
CLOUD_SHDW_CLASSES = np.array([3, 8, 9, 10], dtype=np.uint8)  # clouds + shadow. Add class 2 to include topographic cast shadows.
TARGET_BAND = "B11"  # 20 m

# Build target grid from first item’s 20 m asset
with rio.open(items[0].assets[TARGET_BAND].href) as ds20:
    aoi_ll = features.bounds(area_of_interest)
    aoi_native = warp.transform_bounds("EPSG:4326", ds20.crs, *aoi_ll)
    win = windows.from_bounds(*aoi_native, transform=ds20.transform).round_offsets().round_lengths()
    H, W = win.height, win.width
    dst_transform = ds20.window_transform(win)
    dst_crs = ds20.crs

def reproject_match(src, src_transform, src_crs, dst_transform, dst_crs, out_shape, resampling=enums.Resampling.nearest):
    out = np.zeros(out_shape, dtype=src.dtype)
    warp.reproject(
        source=src, destination=out,
        src_transform=src_transform, src_crs=src_crs,
        dst_transform=dst_transform, dst_crs=dst_crs,
        resampling=resampling, num_threads=2,
    )
    return out

rows = []
gdal_env = dict(
    GDAL_DISABLE_READDIR_ON_OPEN="EMPTY_DIR",
    CPL_VSIL_CURL_USE_HEAD="NO",
    CPL_VSIL_CURL_ALLOWED_EXTENSIONS=".tif,.TIF",
)

for it in items:
    if "SCL" not in it.assets:
        continue
    scl_href = it.assets["SCL"].href

    with rio.Env(**gdal_env):
        with rio.open(scl_href) as ds:
            # window in SCL’s native CRS
            aoi_scl = warp.transform_bounds("EPSG:4326", ds.crs, *aoi_ll)
            win_scl = windows.from_bounds(*aoi_scl, transform=ds.transform).round_offsets().round_lengths()
            scl_u8 = ds.read(1, window=win_scl).astype(np.uint8)
            s_tr = ds.window_transform(win_scl)

    # Reproject to 20 m grid
    scl_match = reproject_match(
        scl_u8, s_tr, ds.crs, dst_transform, dst_crs,
        out_shape=(H, W), resampling=enums.Resampling.nearest
    )

    # Valid pixels: exclude SCL=0 (No data)
    valid = scl_match != 0
    n_valid = int(valid.sum())
    if n_valid == 0:
        frac_cloud = np.nan
        frac_cloud_shadow = np.nan
    else:
        is_cloud = np.isin(scl_match, CLOUD_CLASSES)
        is_cloud_shadow = np.isin(scl_match, CLOUD_SHDW_CLASSES)
        frac_cloud = float(is_cloud.sum()) / n_valid
        frac_cloud_shadow = float(is_cloud_shadow.sum()) / n_valid

    rows.append({
        "date": np.datetime64(it.datetime),
        "frac_cloud": frac_cloud,
        "frac_cloud_shadow": frac_cloud_shadow,
        "eo_cloud_cover": eo.ext(it).cloud_cover  # STAC percent for the full granule (optional)
    })

df = pd.DataFrame(rows).sort_values("date").set_index("date")

# --- Plot ---
fig, ax = plt.subplots(figsize=(10,4))
ax.plot(df.index, df["frac_cloud"], marker="o", markersize=3, label="Cloud fraction (SCL 8/9/10)")
ax.plot(df.index, df["frac_cloud_shadow"], marker="*", markersize=3, linestyle="--", label="Cloud+shadow (SCL 3/8/9/10)")
ax.set_ylim(-0.1, 1.1)
ax.set_ylabel("Cloud fraction")
ax.set_xlabel("Date")
# ax.set_title("Sentinel-2 cloud cover fraction over AOI")
ax.grid(True, alpha=0.3)
ax.legend(fontsize=10)
plt.tight_layout(); plt.show()


#### e) Export PNGs for quick sharing

In [ ]:
# files = sorted(glob.glob("/bsuhome/tnde/scratch/felix/Sentinel-2/s2_albedo_outputs/*_albedoSW_keepAll.tif"))
# for f in files:
#     with rio.open(f) as ds:
#         arr = ds.read(1, masked=True).filled(np.nan)
#     png = (np.clip(arr, 0, 1) * 255).astype(np.uint8)
#     Image.fromarray(np.nan_to_num(png, nan=0)).save(f.replace(".tif", ".png"))


#### g) Side-by-side keep-snow vs snow-only (same date):

In [ ]:
ds = xr.open_dataset("/bsuhome/tnde/scratch/felix/Sentinel-2/s2_albedo_outputs/albedo_stack_merged.nc")
date = "2021-11-27"  # target calendar date
target = np.datetime64(date, 'ns')

# Compute nearest index *by position*
t = ds.time.values.astype("datetime64[ns]")
i = int(np.argmin(np.abs(t - target)))
nearest_time = t[i]

# (optional) enforce a max tolerance (e.g., <= 1 day away)
if np.abs(nearest_time - target) > np.timedelta64(1, "D"):
    raise ValueError(f"No slice within 1 day of {date}. Nearest is {nearest_time}.")

# Select by position (works even with duplicate labels)
da_keep = ds["albedo_sw_keep_hard"].isel(time=i).squeeze(drop=True)
da_snow = ds["albedo_sw_snow_hard"].isel(time=i).squeeze(drop=True)

# Plot
fig, axs = plt.subplots(1, 2, figsize=(12, 5), constrained_layout=True)
# da_keep.plot.imshow(ax=axs[0], vmin=0, vmax=1, add_colorbar=True, extent=extent, origin="upper")
im = axs[0].imshow(np.ma.filled(da_keep, np.nan), vmin=0, vmax=1, extent=extent, origin="upper")
axs[0].set_title(f"{date} • SW albedo • keep-snow")
axs[0].tick_params(axis='x', rotation=0)
# da_snow.plot.imshow(ax=axs[1], vmin=0, vmax=1, add_colorbar=True)
im = axs[1].imshow(np.ma.filled(da_snow, np.nan), vmin=0, vmax=1, extent=extent, origin="upper")
axs[1].set_title(f"{date} • SW albedo • snow-only")
axs[1].tick_params(axis='x', rotation=0)

# Single shared colorbar for all panels
cbar = fig.colorbar(im, ax=axs, fraction=0.046, pad=0.04)
cbar.set_label("Albedo (0–1)")
plt.show()

#### h) GeoTIFF route 

In [ ]:
path = "/bsuhome/tnde/scratch/felix/Sentinel-2/s2_albedo_outputs/2021-11-27_albedoSW_keepAll_hard.tif"
with rio.open(path) as ds_r:
    arr = ds_r.read(1, masked=True)
    extent = plotting_extent(ds_r)

plt.figure(figsize=(8,6))
plt.imshow(arr, extent=extent, origin="upper") #, vmin=0, vmax=1)
plt.colorbar(label="Albedo (0–1)")
plt.title("Shortwave albedo (keep-all)")
plt.tick_params(axis='x', rotation=45)
plt.show()


#### Folium

In [ ]:
tif = "/bsuhome/tnde/scratch/felix/Sentinel-2/s2_albedo_outputs/2021-11-27_albedoSW_keepAll_hard.tif"
png = tif.replace(".tif", ".overlay.png")

with rio.open(tif) as ds:
    arr = ds.read(1, masked=True).astype("float32")
    data = arr.filled(np.nan)
    nodata_mask = np.isnan(data)

    # ----- Choose color range -----
    # Albedo is typically [0, 1]; adjust if the range differs.
    vmin, vmax = 0.0, 1.0
    # Optional: stretch to percentiles instead of fixed bounds
    # lo, hi = np.nanpercentile(data, [2, 98]); vmin, vmax = float(lo), float(hi)

    # ----- Map to colors -----
    cmap = plt.get_cmap("viridis")           # try "turbo", "magma", "plasma", etc.
    norm = colors.Normalize(vmin=vmin, vmax=vmax, clip=True)
    rgba = cmap(norm(data), bytes=True)      # uint8 RGBA, shape (H,W,4)

    # Make NaNs transparent
    rgba[nodata_mask, 3] = 0

    Image.fromarray(rgba, mode="RGBA").save(png)

    # bounds in WGS84 for Folium
    left, bottom, right, top = transform_bounds(ds.crs, "EPSG:4326", *ds.bounds)
    center = [(bottom + top) / 2, (left + right) / 2]

m = folium.Map(location=center, zoom_start=11, tiles="CartoDB Positron")
folium.raster_layers.ImageOverlay(
    name="SW keep 2021-11-27 (colored)",
    image=png,
    bounds=[[bottom, left], [top, right]],
    opacity=0.85,     # overlay opacity (keep some basemap context)
).add_to(m)

# ----- Add a colorbar legend -----
cm = bcm.LinearColormap(
    colors=["#440154", "#31688e", "#35b779", "#fde725"],  # viridis key colors
    vmin=vmin, vmax=vmax, caption="Shortwave Albedo"
)
cm.add_to(m)

folium.LayerControl().add_to(m)
m


In [ ]:
tif = "/bsuhome/tnde/scratch/felix/Sentinel-2/s2_albedo_outputs/2021-11-27_albedoSW_keepAll_soft.tif"
png = tif.replace(".tif", ".overlay.png")

with rio.open(tif) as ds:
    arr = ds.read(1, masked=True).astype("float32")
    data = arr.filled(np.nan)
    nodata_mask = np.isnan(data)

    # ----- Choose color range -----
    # Albedo is typically [0, 1]; adjust if the range differs.
    vmin, vmax = 0.0, 1.0
    # Optional: stretch to percentiles instead of fixed bounds
    # lo, hi = np.nanpercentile(data, [2, 98]); vmin, vmax = float(lo), float(hi)

    # ----- Map to colors -----
    cmap = plt.get_cmap("viridis")           # try "turbo", "magma", "plasma", etc.
    norm = colors.Normalize(vmin=vmin, vmax=vmax, clip=True)
    rgba = cmap(norm(data), bytes=True)      # uint8 RGBA, shape (H,W,4)

    # Make NaNs transparent
    rgba[nodata_mask, 3] = 0

    Image.fromarray(rgba, mode="RGBA").save(png)

    # bounds in WGS84 for Folium
    left, bottom, right, top = transform_bounds(ds.crs, "EPSG:4326", *ds.bounds)
    center = [(bottom + top) / 2, (left + right) / 2]

m = folium.Map(location=center, zoom_start=11, tiles="CartoDB Positron")
folium.raster_layers.ImageOverlay(
    name="SW keep 2021-11-27 (colored)",
    image=png,
    bounds=[[bottom, left], [top, right]],
    opacity=0.85,     # overlay opacity (keep some basemap context)
).add_to(m)

# ----- Add a colorbar legend -----
cm = bcm.LinearColormap(
    colors=["#440154", "#31688e", "#35b779", "#fde725"],  # viridis key colors
    vmin=vmin, vmax=vmax, caption="Shortwave Albedo"
)
cm.add_to(m)

folium.LayerControl().add_to(m)
m


### Build a Simple Interactive Webpage

In [ ]:
# mamba install -c conda-forge folium geopandas rasterio pillow matplotlib
def build_albedo_webmap(
    tiff_dir="/bsuhome/tnde/scratch/felix/Sentinel-2/s2_albedo_outputs",
    out_html="/bsuhome/tnde/scratch/felix/Sentinel-2/s2_albedo_outputs/webmap_sw_keepAll.html",
    product="sw",            # "sw" | "vis" | "nir"  (used only when metric="albedo")
    variant="keep",          # "keep" | "snow"
    metric="albedo",         # "albedo" | "BSA20m" | "WSA20m" | "BLUE20m"
    shapefile="/bsuhome/tnde/scratch/felix/modis/East_River_SHP/ER_bbox.shp",
    vmin=0.0, vmax=1.0,
    opacity=0.85,
    palette="viridis",
    zip_output=True,
):
    """
    Create a shareable Folium HTML with albedo layers + AOI outline.

    - metric="albedo": looks for files like {date}_albedoSW_keepSnow.tif (the original outputs)
      and respects 'product' ("sw|vis|nir").
    - metric in {"BSA20m","WSA20m","BLUE20m"}: looks for {date}_*_{metric}_{keepSnow|snowOnly}.tif
      (the new BRDF-derived outputs). 'product' is ignored for these.
    - Saves HTML and an 'assets' folder of colorized PNGs. If zip_output=True, writes a .zip too.
    """
    metric = metric.strip()
    variant = variant.lower()
    if variant not in {"keep", "snow"}:
        raise ValueError("variant must be 'keep' or 'snow'")

    # Choose filename pattern + labels
    tiff_dir = os.path.abspath(tiff_dir)
    out_html = os.path.abspath(out_html)
    out_dir  = os.path.dirname(out_html)

    # NEW: ensure glob is imported before first use
    import glob
    if metric.lower() == "albedo":
        product = product.lower()
        var_tag = "keepAll_hard" if variant == "keep" else "snowOnly_hard"
        if product not in {"sw", "vis", "nir"}:
            raise ValueError("product must be 'sw'|'vis'|'nir' when metric='albedo'")
        prod_tag = {"sw": "SW", "vis": "VIS", "nir": "NIR"}[product]
        pattern = os.path.join(tiff_dir, f"*_{'albedo'+prod_tag}_{var_tag}.tif")
        layer_prefix = f"{product.upper()} {variant}"
        assets_dir = os.path.join(out_dir, f"web_assets_{product}_{variant}")
        png_stub   = f"{product}_{variant}"
        legend_label = "Albedo (0–1)"
    else:
        if metric not in {"BSA20m", "WSA20m", "BLUE20m"}:
            raise ValueError("metric must be 'albedo' or one of {'BSA20m','WSA20m','BLUE20m'}")
        var_tag = "SW_hard" if variant == "keep" else "snowOnly_hard"
        # Matches e.g. 2021-10-05_S2A_BSA20m_keepSnow.tif or 2021-10-05_s2_fused_BSA20m_keepSnow.tif
        pattern = os.path.join(tiff_dir, f"*_{metric}_{var_tag}.tif")
        layer_prefix = f"{metric} {variant}"
        assets_dir = os.path.join(out_dir, f"web_assets_{metric.lower()}_{variant}")
        png_stub   = f"{metric.lower()}_{variant}"
        legend_label = f"{metric.replace('20m',' 20 m')} (0–1)"

    tifs = sorted(glob.glob(pattern))
    if not tifs:
        raise FileNotFoundError(f"No files found for pattern: {pattern}")

    pathlib.Path(assets_dir).mkdir(parents=True, exist_ok=True)

    # AOI (project to WGS84 for Folium)
    aoi_geojson = None
    if shapefile and os.path.exists(shapefile):
        try:
            gdf = gpd.read_file(shapefile).to_crs("EPSG:4326")
            aoi_geojson = gdf.__geo_interface__
        except Exception as e:
            print(f"[warn] AOI overlay skipped: {e}")

    # Color mapping
    # If `palette` is a string like "viridis", grab it from matplotlib
    # If it's already a colormap object, just use it directly.
    if isinstance(palette, str):
        cmap = mpl_cm.get_cmap(palette)
    else:
        cmap = palette  # assume it's already a Colormap

    norm = colors.Normalize(vmin=vmin, vmax=vmax, clip=True)


    # Build map and bounds
    m = folium.Map(tiles="CartoDB Positron", zoom_start=11, control_scale=True)
    union_bounds = [ +1e9, +1e9, -1e9, -1e9 ]  # left, bottom, right, top

    # Add one overlay per date
    first_layer = True  # <<< show only the earliest (first) layer by default
    for tif in tifs:
        date = os.path.basename(tif).split("_")[0]
        with rio.open(tif) as ds:
            arr = ds.read(1, masked=True).filled(np.nan)

            # RGBA (NaNs transparent)
            rgba = cmap(norm(arr))
            rgba[..., 3] = np.where(np.isfinite(arr), opacity, 0.0)
            png = (rgba * 255).astype(np.uint8)

            png_path = os.path.join(assets_dir, f"{date}_{png_stub}.png")
            Image.fromarray(png, mode="RGBA").save(png_path)

            # bounds in WGS84 for overlay
            left, bottom, right, top = transform_bounds(ds.crs, "EPSG:4326", *ds.bounds, densify_pts=21)
            union_bounds[0] = min(union_bounds[0], left)
            union_bounds[1] = min(union_bounds[1], bottom)
            union_bounds[2] = max(union_bounds[2], right)
            union_bounds[3] = max(union_bounds[3], top)

        folium.raster_layers.ImageOverlay(
            name=f"{layer_prefix} {date}",
            image=png_path,  # absolute path -> embedded as base64
            bounds=[[bottom, left], [top, right]],
            opacity=1.0,     # we encoded alpha already
            interactive=False,
            cross_origin=False,
            show=first_layer,            # <<< only earliest layer is visible initially
        ).add_to(m)
        first_layer = False             # subsequent layers hidden by default

    # AOI outline
    if aoi_geojson:
        folium.GeoJson(
            aoi_geojson,
            name="AOI",
            style_function=lambda x: {"color": "black", "weight": 2, "fillOpacity": 0},
        ).add_to(m)

    # Fit to union of layers
    m.fit_bounds([[union_bounds[1], union_bounds[0]], [union_bounds[3], union_bounds[2]]])

    # Legend
    col = LinearColormap(colors=[cmap(0.0), cmap(0.5), cmap(1.0)], vmin=vmin, vmax=vmax)
    col.caption = legend_label
    col.add_to(m)

    # Layer control
    folium.LayerControl(collapsed=False).add_to(m)

    # Save HTML (and optional zip)
    m.save(out_html)
    print(f"[ok] Web map written to: {out_html}")
    print(f"[ok] Assets in: {assets_dir}")

    if zip_output:
        zip_path = os.path.splitext(out_html)[0] + ".zip"
        import zipfile, glob
        with zipfile.ZipFile(zip_path, "w", compression=zipfile.ZIP_DEFLATED) as z:
            z.write(out_html, arcname=os.path.basename(out_html))
            for png in sorted(glob.glob(os.path.join(assets_dir, "*.png"))):
                z.write(png, arcname=os.path.join(os.path.basename(assets_dir), os.path.basename(png)))
        print(f"[ok] Zip created: {zip_path}")
    return out_html

In [ ]:
# shortwave, keep-snow
tif_file_dir = "/bsuhome/tnde/scratch/felix/Sentinel-2/s2_albedo_outputs"
shapefile_path = "/bsuhome/tnde/scratch/felix/modis/East_River_SHP/ER_bbox.shp"
webmap_path = "/bsuhome/tnde/geoscience/albedo_downscaling/sentinel_2/webmap_html/"
build_albedo_webmap(
    tiff_dir=tif_file_dir,
    out_html=webmap_path+"webmap_sw_keepAll.html",
    product="sw",
    variant="keep",
    metric="albedo",
    shapefile=shapefile_path,
    vmin=0, vmax=1, opacity=0.9, palette="viridis",
    zip_output=True,  # also makes webmap_sw_keep.zip for sending
)

# shortwave, snow-only
build_albedo_webmap(
    tiff_dir=tif_file_dir,
    out_html=webmap_path+"webmap_sw_snowOnly.html",
    product="sw",
    variant="snow",
    metric="albedo",
    shapefile=shapefile_path,
)

# VIS keep
build_albedo_webmap(
    tiff_dir=tif_file_dir,
    out_html=webmap_path+"webmap_vis_keepAll.html",
    product="vis",
    variant="keep",
    metric="albedo",
)

# NIR keep
build_albedo_webmap(
    tiff_dir=tif_file_dir,
    out_html=webmap_path+"webmap_nir_keepAll.html",
    product="nir",
    variant="keep",
    metric="albedo",
)


# New BRDF products (broadband SW):
build_albedo_webmap(metric="BSA20m", variant="keep",
                    out_html=webmap_path+"webmap_bsa_keepAll.html")
build_albedo_webmap(metric="WSA20m", variant="snow",
                    out_html=webmap_path+"webmap_wsa_snowOnly.html")
build_albedo_webmap(metric="BLUE20m", variant="keep",
                    out_html=webmap_path+"webmap_blue_keepAll.html")

### Adding Date Dropdown to Webpage

In [ ]:
# mamba install -c conda-forge folium geopandas rasterio pillow matplotlib jinja2
class DateDropdown(MacroElement):
    """Leaflet control with a <select> that toggles date layers."""
    def __init__(self, date_layers, default_date=None):
        """
        date_layers: list of (date_str, layer_js_varname)
        default_date: "earliest" | "latest" | "YYYY-MM-DD" | None
        """
        super().__init__()
        # ensure chronological order
        self.date_layers = sorted(date_layers, key=lambda x: x[0])
        self.default_date = default_date
        self._name = "DateDropdown"
        self._template = Template(u"""
        {% macro script(this, kwargs) %}
        var map = {{ this._parent.get_name() }};
        var dateLayers = {};
        var dates = [];
        {% for date, lname in this.date_layers %}
          dateLayers["{{date}}"] = {{ lname }};
          dates.push("{{date}}");
        {% endfor %}

        var init = "{{ this.default_date if this.default_date is not none else '' }}";
        if (!init) { init = dates[0]; }
        else if (init === "earliest") { init = dates[0]; }
        else if (init === "latest")   { init = dates[dates.length - 1]; }
        else if (dates.indexOf(init) === -1) { init = dates[0]; }

        var ctrl = L.control({position: 'topright'});
        ctrl.onAdd = function(map){
          var div = L.DomUtil.create('div', 'leaflet-bar');
          div.style.background = 'white';
          div.style.padding = '6px';
          div.style.boxShadow = '0 1px 5px rgba(0,0,0,0.4)';
          var label = L.DomUtil.create('div', '', div);
          label.innerHTML = '<b>Date</b>';
          var select = L.DomUtil.create('select', '', div);
          select.style.minWidth = '150px';
          for (var i=0;i<dates.length;i++){
            var opt = document.createElement('option'); opt.value = dates[i]; opt.text = dates[i];
            select.appendChild(opt);
          }
          L.DomEvent.disableClickPropagation(div);

          function showOnly(sel){
            for (var d in dateLayers){
              if (map.hasLayer(dateLayers[d])) map.removeLayer(dateLayers[d]);
            }
            if (dateLayers[sel]) map.addLayer(dateLayers[sel]);
          }

          select.value = init;
          showOnly(init);
          select.onchange = function(e){ snowOnly(e.target.value); };
          return div;
        };
        ctrl.addTo(map);
        {% endmacro %}
        """)

def build_albedo_webmap_with_dropdown(
    tiff_dir="/bsuhome/tnde/scratch/felix/Sentinel-2/s2_albedo_outputs",
    out_html="/bsuhome/tnde/scratch/felix/Sentinel-2/s2_albedo_outputs/webmap_sw_keepAll.html",
    product="sw",                  # used only when metric="albedo"   -> "sw" | "vis" | "nir"
    variant="keep",                # "keep" | "snow"
    metric="albedo",               # "albedo" | "BSA20m" | "WSA20m" | "BLUE20m"
    shapefile="/bsuhome/tnde/scratch/felix/modis/East_River_SHP/ER_bbox.shp",
    vmin=0.0, vmax=1.0, opacity=0.85, palette="viridis",
    include_layer_control=False,
    zip_output=True,
    default_date="earliest",
):
    """
    Create a Folium HTML with a date dropdown.
    - metric="albedo": looks for {date}_albedo{SW|VIS|NIR}_{keepAll|snowOnly}.tif  (uses 'product')
    - metric in {"BSA20m","WSA20m","BLUE20m"}: looks for {date}_*_{metric}_{keepAll|snowOnly}.tif
      (e.g., 2021-10-05_S2A_BSA20m_keepAll.tif). 'product' is ignored.
    """
    variant = variant.lower()
    if variant not in {"keep","snow"}:
        raise ValueError("variant must be 'keep' or 'snow'")

    metric = metric.strip()
    tiff_dir = os.path.abspath(tiff_dir)
    out_html = os.path.abspath(out_html)
    out_dir  = os.path.dirname(out_html)

    if metric.lower() == "albedo":
        product = product.lower()
        var_tag = "keepAll_hard" if variant == "keep" else "snowOnly_hard"
        if product not in {"sw","vis","nir"}:
            raise ValueError("When metric='albedo', product must be 'sw'|'vis'|'nir'")
        prod_tag = {"sw":"SW","vis":"VIS","nir":"NIR"}[product]
        pattern = os.path.join(tiff_dir, f"*_{'albedo'+prod_tag}_{var_tag}.tif")
        layer_prefix = f"{product.upper()} {variant}"
        assets_dir = os.path.join(out_dir, f"web_assets_{product}_{variant}")
        png_stub   = f"{product}_{variant}"
        legend_label = "Albedo (0–1)"
    else:
        if metric not in {"BSA20m","WSA20m","BLUE20m"}:
            raise ValueError("metric must be 'albedo' or one of {'BSA20m','WSA20m','BLUE20m'}")
        var_tag = "SW_hard" if variant == "keep" else "snowOnly_hard"
        pattern = os.path.join(tiff_dir, f"*_{metric}_{var_tag}.tif")
        layer_prefix = f"{metric} {variant}"
        assets_dir = os.path.join(out_dir, f"web_assets_{metric.lower()}_{variant}")
        png_stub   = f"{metric.lower()}_{variant}"
        legend_label = f"{metric.replace('20m',' 20 m')} (0–1)"

    tifs = sorted(glob.glob(pattern))
    if not tifs:
        raise FileNotFoundError(f"No files found for pattern: {pattern}")

    pathlib.Path(assets_dir).mkdir(parents=True, exist_ok=True)

    # AOI outline (WGS84)
    aoi_geojson = None
    if shapefile and os.path.exists(shapefile):
        try:
            aoi_geojson = gpd.read_file(shapefile).to_crs("EPSG:4326").__geo_interface__
        except Exception as e:
            print(f"[warn] AOI overlay skipped: {e}")

    cmap = cm.get_cmap(palette)
    norm = colors.Normalize(vmin=vmin, vmax=vmax, clip=True)

    m = folium.Map(tiles="CartoDB Positron", zoom_start=11, control_scale=True)
    union = [+1e9, +1e9, -1e9, -1e9]
    date_layers = []

    for tif in tifs:
        date = os.path.basename(tif).split("_")[0]
        with rio.open(tif) as ds:
            arr = ds.read(1, masked=True).filled(np.nan)
            rgba = cmap(norm(arr))
            rgba[..., 3] = np.where(np.isfinite(arr), opacity, 0.0)
            png = (rgba * 255).astype(np.uint8)
            png_path = os.path.join(assets_dir, f"{date}_{png_stub}.png")
            Image.fromarray(png, mode="RGBA").save(png_path)

            left, bottom, right, top = transform_bounds(ds.crs, "EPSG:4326", *ds.bounds, densify_pts=21)
            union[0] = min(union[0], left);  union[1] = min(union[1], bottom)
            union[2] = max(union[2], right); union[3] = max(union[3], top)

        overlay = folium.raster_layers.ImageOverlay(
            name=f"{layer_prefix} {date}",
            image=png_path,  # absolute path -> embedded as base64
            bounds=[[bottom, left], [top, right]],
            opacity=1.0,
            interactive=False,
            cross_origin=False,
        )
        overlay.add_to(m)
        date_layers.append((date, overlay.get_name()))

    # AOI outline
    if aoi_geojson:
        folium.GeoJson(
            aoi_geojson, name="AOI",
            style_function=lambda x: {"color":"black","weight":2,"fillOpacity":0},
        ).add_to(m)

    # Fit and legend
    m.fit_bounds([[union[1], union[0]], [union[3], union[2]]])
    col = LinearColormap(colors=[cmap(0.0), cmap(0.5), cmap(1.0)], vmin=vmin, vmax=vmax)
    col.caption = legend_label
    col.add_to(m)

    if include_layer_control:
        folium.LayerControl(collapsed=True).add_to(m)

    m.add_child(DateDropdown(date_layers, default_date=default_date))

    m.save(out_html)
    print(f"[ok] Web map with dropdown written to: {out_html}")

    if zip_output:
        zip_path = os.path.splitext(out_html)[0] + ".zip"
        with zipfile.ZipFile(zip_path, "w", compression=zipfile.ZIP_DEFLATED) as z:
            z.write(out_html, arcname=os.path.basename(out_html))
            for png in sorted(glob.glob(os.path.join(assets_dir, "*.png"))):
                z.write(png, arcname=os.path.join(os.path.basename(assets_dir), os.path.basename(png)))
        print(f"[ok] Zip created: {zip_path}")
    return out_html

In [ ]:
# Original NTB products
build_albedo_webmap_with_dropdown(
    tiff_dir="/bsuhome/tnde/scratch/felix/Sentinel-2/s2_albedo_outputs",
    out_html=webmap_path+"webmap_sw_keepAll_date_dropdown.html",
    product="sw",
    variant="keep",
    metric="albedo",
    shapefile=shapefile_path,
    vmin=0, vmax=1, opacity=0.9, palette="viridis",
    include_layer_control=False, # dropdown only
    zip_output=True,             # also creates .zip for emailing
    default_date="earliest", # select earliest date. use "latest" for most recent date or pass "YYYY-MM-DD" for a specific date
)

# New BRDF products (broadband SW)
build_albedo_webmap_with_dropdown(metric="BSA20m", variant="keep", 
                                  out_html=webmap_path+"webmap_bsa_keepAll_date_dropdown.html")

build_albedo_webmap_with_dropdown(metric="WSA20m", variant="snow", 
                                  out_html=webmap_path+"webmap_wsa_snowOnly_date_dropdown.html")

build_albedo_webmap_with_dropdown(metric="BLUE20m", variant="keep", 
                                  out_html=webmap_path+"webmap_blue_keepAll_date_dropdown.html")